In [21]:
# Lijstje van de benodigde imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import math

# Datumfilter instellen
start_datum = '2021-09-01'
eind_datum = '2021-12-01'

# Invoer parameters
# Begintijd
t0 = 0                          # Beginwaarde tijd
# Bergend oppervlak
A_hellend_dak = 660.88          # (m^2)
A_vlak_dak = 56.12              # (m^2)
A_rijweg_goot = 836.99          # (m^2)
A_trottoir_goot = 532.36        # (m^2)
A_parkeerplaatsen = 570.08      # (m^2)
# Rockflow
A_RF = 1337.53                  # Oppervlak van het bergend pakket (m^2)
k_RF = 200                      # Doorlatendheidscoëfficiënt van de rockflow (m/dag)
p_RF = 0.95                     # Porositeit rockflow (-)
Onderkant_RF = -1.08            # (mNAP)
# Drainzand
k_DZ = 10                       # Doorlatendheidscoëfficiënt van het drainzand (m/dag)
p_DZ = 0.25                     # Porositeit drainzand (-)
B_DZ_B = 0.70                   # Breedte bovenkant drainzand (m)
B_DZ_O = 0.30                   # Breedte onderkant drainzand (m)
L_DZ = 180                      # Lengte onderkant drainzand, evenals de weg (m)
Onderkant_DZ = -1.70            # (mNAP), correctie van 5cm door rechttrekken lijn voor gemiddelde tussen -1,65 en -1,75 mNAP
# Weg
B_W = 5.85                      # Breedte van de weg (m)
# Klei
k_KL = 0.01                     # Doorlatendheidscoëfficiënt van de bodem (m/dag)
p_KL = 0.4                      # Porositeit klei (-)
# Drain
Diameter_DR = 0.1               # Diameter drainagebuis (m) 

# Berekenen parameters
# Bergend oppervlak
A_afstromend = (A_hellend_dak + A_vlak_dak + A_rijweg_goot + A_trottoir_goot + A_parkeerplaatsen) # (m^2)
# Rockflow
k_RF = k_RF/24/12               # Doorlatendheidscoëfficiënt van de rockflow (m/5min)
# Drainzand
A_DZ = B_DZ_B * L_DZ            # Oppervlak van het drainzand (m^2)
k_DZ = k_DZ/24/12               # Doorlatendheidscoëfficiënt van het drainzand (m/5min)
VMAX_DZ = 0.3348 * L_DZ * p_DZ  # Maximaal volume in het drainzand (m^3)
# Klei
k_KL = k_KL/24/12               # Doorlatendheidscoëfficiënt van de klei (m/5min)
# Drain
A_DR = (math.pi/4)*Diameter_DR**2   # Oppervlak drain (m^2)

# Begin rekenen code; Omzetten verkregen dataset KNMI naar lijst met gefilterde waardes
# Inladen neerslagdata
df_neerslag = pd.read_csv('Neerslag_Hoogmade.csv', skipinitialspace=True, header=0, sep=';')
# Aanmaken van een datum kolom
df_neerslag['DateTime'] = pd.to_datetime(df_neerslag['datum'])
# Zet 'neerslagintensiteit (mm/uur)' om naar numerieke waarden
df_neerslag['neerslagintensiteit (mm/uur)'] = pd.to_numeric(df_neerslag['neerslagintensiteit (mm/uur)'], errors='coerce')
# Vervangen NaN-waardes door 0
df_neerslag['neerslagintensiteit (mm/uur)'].fillna(0, inplace=True)
# Selecteren van de rijen binnen het gewenste datum bereik
df_filtered = df_neerslag[(df_neerslag['DateTime'] >= start_datum) & (df_neerslag['DateTime'] <= eind_datum)]

# Uitschakelen van de waarschuwing
pd.set_option('mode.chained_assignment', None)
# Omzetten van de neerslag van mm/uur naar m/uur (delen door 1000), daarna van uur naar 5min (delen door 12)
df_filtered['neerslagintensiteit (mm/uur)'] = df_filtered['neerslagintensiteit (mm/uur)'] / 1000 / 12
# Inschakelen van de waarschuwing
pd.set_option('mode.chained_assignment', 'warn')
# Neerslagwaardes toevoegen aan lijst
neerslag_lijst = df_filtered['neerslagintensiteit (mm/uur)'].tolist()

# Arrays
dt = 1                                      # Tijdstap (5 minuten)
t = np.arange(0, len(neerslag_lijst), dt)   # Array voor de tijdstappen 
V_RF = np.zeros(len(t))                     # Array voor het volume water in de rockflow (m^3)
H_RF = np.zeros(len(t))                     # Array voor de waterhoogte in de rockflow (mNAP)
V_DZ = np.zeros(len(t))                     # Array voor het volume water in het drainzand (m^3)
H_DZ = np.zeros(len(t))                     # Array voor de waterhoogte in het drainzand (mNAP)

# Initiële conditie
V_RF[0] = 0.0                               # Initiële volume water in het rockflow (m^3)
V_DZ[0] = 0.0                               # Initiële volume water in het drainzand (m^3)

# Berekenen waterstroming in waterbergende weg
for i in range(1, len(t)):  
    if H_RF[i-1] > -1.08:
        Q_uit_RF_KL = (L_DZ * 2 * (H_RF[i-1] - Onderkant_RF) + L_DZ * (B_W - B_DZ_B)) * k_KL
    else:
        Q_uit_RF_KL = 0.0

    Q_in_RF = neerslag_lijst[i] * A_afstromend 
    Q_uit_RF = min(V_RF[i-1], A_DZ * k_DZ, VMAX_DZ - V_DZ[i-1]) + Q_uit_RF_KL
  
    if H_RF[i-1] >= -1.0:
        Q_uit_DR = 2 * A_DR * (math.sqrt(2*9.81*(H_RF[i-1]--1))) * 300 + ((L_DZ * 2 * ((H_DZ[i-1] -(Onderkant_DZ))/ math.cos(math.radians(25)))+ (L_DZ * B_DZ_O))) * k_KL
    else:
        Q_uit_DR = 0.0 + ((L_DZ * 2 * ((H_DZ[i-1] -(Onderkant_DZ))/ math.cos(math.radians(25)))+ (L_DZ * B_DZ_O))) * k_KL
    
    Q_in_DZ = min(V_RF[i-1], A_DZ * k_DZ, VMAX_DZ - V_DZ[i-1])
    Q_uit_DZ = min(V_DZ[i-1], Q_uit_DR) 

    V_RF[i] = (Q_in_RF - Q_uit_RF) * dt + V_RF[i-1]
    H_RF[i] = V_RF[i] / (p_RF * A_RF) + Onderkant_RF
    V_DZ[i] = (Q_in_DZ - Q_uit_DZ) * dt + V_DZ[i-1]
    H_DZ[i] = V_DZ[i] / (p_DZ * A_DZ) + Onderkant_DZ

# Afdrukken van waarden op index 515 afgerond op twee decimalen
Specifieke_waarde = 5866

print("Specifieke waarde")
print("V_RF:", round(V_RF[Specifieke_waarde], 2))
print("H_RF:", round(H_RF[Specifieke_waarde], 2))
print("V_DZ:", round(V_DZ[Specifieke_waarde], 2))
print("H_DZ:", round(H_DZ[Specifieke_waarde], 2))



Specifieke waarde
V_RF: 47.75
H_RF: -1.04
V_DZ: 15.06
H_DZ: -1.22
